In [9]:
suppressPackageStartupMessages({
    library(DESeq2)
    library(dplyr)
})

Add time points and cell types
deploying shinyapp on aws

In [10]:
# URL of the compressed file
url <- "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE179nnn/GSE179487/suppl/GSE179487_FINAL_master_list_of_gene_counts_MIN.Fastqs.txt.gz"

# Download the compressed file to a temporary location
temp_compressed <- tempfile(fileext = ".gz")
download.file(url, temp_compressed)

# Decompress the file
temp_decompressed <- tempfile()
gzfile <- gzfile(temp_compressed, "rb")
writeLines(readLines(gzfile), temp_decompressed)
close(gzfile)

In [11]:
# Counts
counts <- read.table(temp_decompressed, header = TRUE)
rownames(counts) <- make.unique(counts$geneSymbol)
counts$geneSymbol <- NULL
counts$id <- NULL
counts$geneCoordinate <- NULL
colnames(counts) <- sub(pattern = "^X", replacement = "", colnames(counts))
counts_mat <- as.matrix(counts)
head(counts_mat)

,19616.021_HiHi_oW_S17,19616.022_ABC_oW_S20,19616.022_HiHi_oW_S21,19616.022_naiveB_oW_S19,19616.023_HiHi_bL_S15,19616.023_HiHi_oW_S18,19616.025_ABC_oW_S27,19616.025_HiHi_oW_S29,19616.025_naiveB_oW_S26,19616.025_PB_oW_S28,⋯,FS1819.131_ABC_bL_S2,FS1819.131_HiHi_bL_S3,FS1819.131_HiHi_oW_S10,FS1819.131_naiveB_bL_S1,FS1819.131_naiveB_oW_S8,FS1819.131_PB_oW_S9,FS1819.999_ABC_bL_S5,FS1819.999_HiHi_bL_S7,FS1819.999_naiveB_bL_S4,FS1819.999_PB_bL_S6
TSPAN6,0,0,0,0,0,1,0,2,0,3,⋯,2,4,17,1,3,1,0,1,0,0
DPM1,191,113,181,87,221,163,117,142,86,114,⋯,161,168,99,100,91,85,92,179,94,137
SCYL3,8,21,21,8,52,24,38,18,19,19,⋯,60,16,10,14,16,26,53,19,13,14
C1orf112,5,1,3,4,16,7,8,5,0,5,⋯,1,4,0,2,4,0,4,2,0,13
FGR,125,299,51,457,57,0,794,476,78,637,⋯,330,108,722,433,492,695,233,303,338,495
CFH,0,0,2,0,0,3,0,1,0,0,⋯,0,0,0,1,0,6,0,1,0,0


In [12]:
# Metadata
metadata <- read.delim("metadata.txt")
head(metadata)

,id,Sample_title,Sample_geo_accession,Time_Point,condition,Cell_Type
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,19616-021_HiHi_oW_S17,anti-PD1-treated adult 1_HiHi_oW,GSM5419198,1 Week,anti-PD1,ICOS+CD38+ cTfh
2,19616-022_ABC_oW_S20,anti-PD1-treated adult 2_ABC_oW,GSM5419199,1 Week,anti-PD1,Activated B cells
3,19616-022_HiHi_oW_S21,anti-PD1-treated adult 2_HiHi_oW,GSM5419200,1 Week,anti-PD1,ICOS+CD38+ cTfh
4,19616-022_naiveB_oW_S19,anti-PD1-treated adult 2_naiveB_oW,GSM5419201,1 Week,anti-PD1,Naïve B cells
5,19616-023_HiHi_bL_S15,anti-PD1-treated adult 3_HiHi_bL,GSM5419202,Baseline,anti-PD1,ICOS+CD38+ cTfh
6,19616-023_HiHi_oW_S18,anti-PD1-treated adult 3_HiHi_oW,GSM5419203,1 Week,anti-PD1,ICOS+CD38+ cTfh


In [13]:
# DEG
dds <- suppressWarnings(DESeqDataSetFromMatrix(countData = counts_mat,
                              colData = metadata,
                              design = ~ condition))

dds <- suppressWarnings(DESeq(dds))
dds <- estimateSizeFactors(dds)
analysis <- counts(dds, normalized=TRUE)

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning 

In [54]:
head(analysis)

,19616.021_HiHi_oW_S17,19616.022_ABC_oW_S20,19616.022_HiHi_oW_S21,19616.022_naiveB_oW_S19,19616.023_HiHi_bL_S15,19616.023_HiHi_oW_S18,19616.025_ABC_oW_S27,19616.025_HiHi_oW_S29,19616.025_naiveB_oW_S26,19616.025_PB_oW_S28,⋯,FS1819.131_ABC_bL_S2,FS1819.131_HiHi_bL_S3,FS1819.131_HiHi_oW_S10,FS1819.131_naiveB_bL_S1,FS1819.131_naiveB_oW_S8,FS1819.131_PB_oW_S9,FS1819.999_ABC_bL_S5,FS1819.999_HiHi_bL_S7,FS1819.999_naiveB_bL_S4,FS1819.999_PB_bL_S6
TSPAN6,0.000000,0.0000000,0.000000,0.000000,0.00000,0.7914183,0.000000,1.7922051,0.00000,3.262879,⋯,1.9516956,3.317843,18.50219,0.9714045,3.091778,1.086146,0.000000,0.825554,0.00000,0.00000
DPM1,164.127376,104.1528300,157.302135,91.974569,184.52080,129.0011783,131.148482,127.2465631,90.62401,123.989416,⋯,157.1114962,139.349423,107.74802,97.1404538,93.783921,92.322410,82.909385,147.774158,92.74952,136.01555
SCYL3,6.874445,19.3558357,18.250524,8.457432,43.41666,18.9940385,42.595233,16.1298460,20.02158,20.664903,⋯,58.5508681,13.271374,10.88364,13.5996635,16.489481,28.239796,47.763015,15.685525,12.82706,13.89940
C1orf112,4.296528,0.9217065,2.607218,4.228716,13.35897,5.5399279,8.967418,4.4805128,0.00000,5.438132,⋯,0.9758478,3.317843,0.00000,1.9428091,4.122370,0.000000,3.604756,1.651108,0.00000,12.90659
FGR,107.413204,275.5902316,44.322701,483.130783,47.59134,0.0000000,890.016191,426.5448173,82.19387,692.818053,⋯,322.0297748,89.581772,785.79870,420.6181651,507.051531,754.871467,209.977029,250.142848,333.50360,491.44306
CFH,0.000000,0.0000000,1.738145,0.000000,0.00000,2.3742548,0.000000,0.8961026,0.00000,0.000000,⋯,0.0000000,0.000000,0.00000,0.9714045,0.000000,6.516876,0.000000,0.825554,0.00000,0.00000


In [16]:
phen <- tibble::rownames_to_column(as.data.frame(colData(dds)), "Sample") %>%
    rename(Condition = condition) %>%
    mutate(Source = 'PBMC') %>%
    select(Sample, Condition, Time_Point, Cell_Type, Source) %>%
    mutate(across(everything(), as.character))
head(phen)

,Sample,Condition,Time_Point,Cell_Type,Source
,<chr>,<chr>,<chr>,<chr>,<chr>
1,19616.021_HiHi_oW_S17,anti-PD1,1 Week,ICOS+CD38+ cTfh,PBMC
2,19616.022_ABC_oW_S20,anti-PD1,1 Week,Activated B cells,PBMC
3,19616.022_HiHi_oW_S21,anti-PD1,1 Week,ICOS+CD38+ cTfh,PBMC
4,19616.022_naiveB_oW_S19,anti-PD1,1 Week,Naïve B cells,PBMC
5,19616.023_HiHi_bL_S15,anti-PD1,Baseline,ICOS+CD38+ cTfh,PBMC
6,19616.023_HiHi_oW_S18,anti-PD1,1 Week,ICOS+CD38+ cTfh,PBMC


In [17]:
saveRDS(analysis, file = file.path("PBMC_RNAseq_GSE179487", "expression.rds"))
saveRDS(phen, file = file.path("PBMC_RNAseq_GSE179487", "metadata.rds"))

In [53]:
system("R -e \"shiny::runApp('PBMC_RNAseq_GSE179487')\"")